## imports

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import os
from os.path import join, exists
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.autograd import Variable as V
from torch.nn import functional as F
import torch.nn  as nn
from PIL import Image
from IPython.core.debugger import set_trace
import nethook as nethook
import scipy.io as sio
from tqdm import tqdm
import time

## hyperparameters

In [37]:
arch = 'alexnet'
imageset = 'mc8-lummatched'

## directory management

In [38]:
home_dir = '/home/jacobpri/git/RotspaceIT/'

# target directory for saving activations
activation_savedir = join(home_dir,'data','d02_modeling','activations')

# folder of trained model weights
weight_dir = join(home_dir,'data','d02_modeling','weights')

# folder of image sets
imageset_dir = join(home_dir,'imagesets')

dirs = os.listdir(imageset_dir)

found = False
for d in dirs:
    d_ = os.listdir(join(imageset_dir,d))
    if (imageset in d_):
        imageset_dir = join(imageset_dir, d, imageset)
        activation_savedir = join(activation_savedir, arch, d, imageset)
        found = True
        break

if found is False:
    raise ValueError('image set not found')   

# create savedir if it doesn't yet exist 
os.makedirs(activation_savedir, exist_ok=True)
print("imageset dir %s" % imageset_dir)
print("weight dir %s" % weight_dir)
print("activation save destination: %s" % activation_savedir)

assert(exists(activation_savedir))
assert(exists(imageset_dir))
assert(exists(weight_dir))

imageset dir /home/jacobpri/git/RotspaceIT/imagesets/experimental/mc8-lummatched
weight dir /home/jacobpri/git/RotspaceIT/data/d02_modeling/weights
activation save destination: /home/jacobpri/git/RotspaceIT/data/d02_modeling/activations/alexnet/experimental/mc8-lummatched


## load architecture

In [39]:
if arch == 'alexnet':
    model = models.alexnet(pretrained = True)
elif arch == 'alexnet-random':
    model = models.alexnet()
elif arch == 'alexnet-object':
    checkpoint = torch.load(os.path.join(weight_dir,'alexnet_imagenet_final.pth.tar'),map_location='cpu')
    checkpoint['state_dict'] = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model = models.alexnet(num_classes=1000)
    model.load_state_dict(checkpoint['state_dict'])
elif arch == 'alexnet-face':
    #model = models.__dict__['alexnet'](num_classes=9131)
    checkpoint = torch.load(os.path.join(weight_dir,'alexnet_faces_final.pth.tar'),map_location='cpu')
    #print(checkpoint['state_dict'].keys())
    checkpoint['state_dict'] = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model = models.alexnet(num_classes=3372)
    model.load_state_dict(checkpoint['state_dict'])
elif arch == 'alexnet-scene':
    checkpoint = torch.load(os.path.join(weight_dir,'alexnet_places_final.pth.tar'),map_location='cpu')
    checkpoint['state_dict'] = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model = models.alexnet(num_classes=365)
    model.load_state_dict(checkpoint['state_dict'])
else:
    raise ValueError('model name not recognized')

print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## get string names/indices of each layer 

In [255]:
model2 = models.densenet161()

In [256]:
list(model2.named_children())

[('features', Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(

In [257]:
model_ = model2

lay_names_torch = []
lay_names_user = []
lay_conv_strs = []
layers = list(model_.named_modules())
count = 0
conv_count = 0

for i in range(0,len(layers)):
    #print(i)
    module_type = layers[i][1].__class__.__name__
    
    nonbasic_types = np.array(['Sequential','BasicBlock','Bottleneck','Fire',
                             '_DenseBlock', '_DenseLayer', 'Transition','InvertedResidual','_InvertedResidual','ConvBNReLU'])
    
    conv_types = ['conv','linear','fc']
    conv_types = ['Conv','Linear']
    
    if np.logical_not(np.isin(module_type, nonbasic_types)):
        lay_names_torch.append(layers[i][0])
        lay_names_user.append(str(count) + '_' + str(layers[i][1]).split('(')[0])

        if any(s in lay_names_user[-1] for s in conv_types) and 'downsample' not in lay_names_torch[-1]:
            conv_count += 1
        lay_conv_strs.append(str(conv_count))

        count += 1
        

In [258]:
print(lay_names_torch)

['', 'features.conv0', 'features.norm0', 'features.relu0', 'features.pool0', 'features.denseblock1.denselayer1.norm1', 'features.denseblock1.denselayer1.relu1', 'features.denseblock1.denselayer1.conv1', 'features.denseblock1.denselayer1.norm2', 'features.denseblock1.denselayer1.relu2', 'features.denseblock1.denselayer1.conv2', 'features.denseblock1.denselayer2.norm1', 'features.denseblock1.denselayer2.relu1', 'features.denseblock1.denselayer2.conv1', 'features.denseblock1.denselayer2.norm2', 'features.denseblock1.denselayer2.relu2', 'features.denseblock1.denselayer2.conv2', 'features.denseblock1.denselayer3.norm1', 'features.denseblock1.denselayer3.relu1', 'features.denseblock1.denselayer3.conv1', 'features.denseblock1.denselayer3.norm2', 'features.denseblock1.denselayer3.relu2', 'features.denseblock1.denselayer3.conv2', 'features.denseblock1.denselayer4.norm1', 'features.denseblock1.denselayer4.relu1', 'features.denseblock1.denselayer4.conv1', 'features.denseblock1.denselayer4.norm2',

In [259]:
for i in range(1,len(lay_names_torch)):
    #num_idx[i][0] += 1
    print((lay_names_torch[i],lay_names_user[i],lay_conv_strs[i]))

#print(lay_types)
#print(len(lay_names))

('features.conv0', '1_Conv2d', '1')
('features.norm0', '2_BatchNorm2d', '1')
('features.relu0', '3_ReLU', '1')
('features.pool0', '4_MaxPool2d', '1')
('features.denseblock1.denselayer1.norm1', '5_BatchNorm2d', '1')
('features.denseblock1.denselayer1.relu1', '6_ReLU', '1')
('features.denseblock1.denselayer1.conv1', '7_Conv2d', '2')
('features.denseblock1.denselayer1.norm2', '8_BatchNorm2d', '2')
('features.denseblock1.denselayer1.relu2', '9_ReLU', '2')
('features.denseblock1.denselayer1.conv2', '10_Conv2d', '3')
('features.denseblock1.denselayer2.norm1', '11_BatchNorm2d', '3')
('features.denseblock1.denselayer2.relu1', '12_ReLU', '3')
('features.denseblock1.denselayer2.conv1', '13_Conv2d', '4')
('features.denseblock1.denselayer2.norm2', '14_BatchNorm2d', '4')
('features.denseblock1.denselayer2.relu2', '15_ReLU', '4')
('features.denseblock1.denselayer2.conv2', '16_Conv2d', '5')
('features.denseblock1.denselayer3.norm1', '17_BatchNorm2d', '5')
('features.denseblock1.denselayer3.relu1', '1

## convert to instrumented model

## retain all layers